<a href="https://colab.research.google.com/github/marziabil/NLP-harrypotter/blob/main/Harry_Potter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.0 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Get text files from drive folder

In [3]:
import os

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

import nltk
nltk.download('punkt')

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the folder containing your text files on Google Drive
folder_path = '/content/drive/MyDrive/NLP analysis'

# Define the desired order of labels
label_order = {
    'Book 1 - The Philosopher\'s Stone.txt': 1,
    'Book 2 - The Chamber of Secrets.txt': 2,
    'Book 3 - The Prisoner of Azkaban.txt': 3,
    'Book 4 - The Goblet of Fire.txt': 4,
    'Book 5 - The Order of the Phoenix.txt': 5,
    'Book 6 - The Half Blood Prince.txt': 6,
    'Book 7 - The Deathly Hallows.txt': 7
}

# Prepare the training data by reading the text files
texts = []
labels = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.txt'):
        with open(file_path, 'r') as f:
            text = f.read()
            texts.append(text)
            label = label_order.get(file_name)
            if label is not None:
                labels.append(label)

                print(f"File: {file_name}, Label: {label}")
                print("Text content (first 5 sentences):")
                sentences = nltk.sent_tokenize(text)
                print('\n'.join(sentences[:2]))
                print("-------------------------------------------")

preprocessing and training the model

In [5]:
# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
num_labels = 3  # Number of sentiment labels (e.g., positive, negative, neutral)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [6]:
# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)  # Assuming three sentiment classifications

# Preprocess and tokenize the text data
def preprocess_text(text):
    return ' '.join(text.strip().split())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Sentiment analysis

In [8]:
import torch.nn.functional as F

# Perform sentiment analysis for each book
for i in range(len(texts)):
    book_title = os.listdir(folder_path)[i]
    book_text = texts[i]
    
    preprocessed_text = preprocess_text(book_text)
    input_ids = tokenizer.encode(preprocessed_text, add_special_tokens=True, truncation=True, max_length=512)
    
    # Convert the input to a PyTorch tensor
    inputs = torch.tensor([input_ids])
    
    # Perform sentiment analysis
    outputs = model(inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
    
    # Apply softmax to get the label probabilities
    probabilities = F.softmax(logits, dim=1)
    sentiment_scores = probabilities.squeeze().tolist()
    
    # Print the book title, predicted sentiment label, and sentiment scores
    sentiment_labels = ['Negative', 'Neutral', 'Positive']
    print(f"Book: {book_title}")
    print(f"Predicted Sentiment: {sentiment_labels[predicted_label]}")
    print(f"Sentiment Scores: {sentiment_scores}")
    print()

Book: Book 1 - The Philosopher's Stone.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.22666002810001373, 0.34922096133232117, 0.4241189956665039]

Book: Book 2 - The Chamber of Secrets.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.21364235877990723, 0.29692456126213074, 0.4894331395626068]

Book: Book 3 - The Prisoner of Azkaban.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.20977291464805603, 0.3549541234970093, 0.4352729916572571]

Book: Book 4 - The Goblet of Fire.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.21579822897911072, 0.35678449273109436, 0.4274172782897949]

Book: Book 5 - The Order of the Phoenix.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.28712013363838196, 0.2894188463687897, 0.42346107959747314]

Book: Book 6 - The Half Blood Prince.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.19611746072769165, 0.3412269949913025, 0.46265557408332825]

Book: Book 7 - The Deathly Hallows.txt
Predicted Sentiment: Positive
Sentiment

## sentiment lexicons

In [14]:
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [15]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the folder containing your text files on Google Drive



# Load SentiWordNet
swn_path = '/content/drive/MyDrive/NLP analysis/SentiWordNet_3.0.0.txt'  # Path to the SentiWordNet text file
swn_entries = {}


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
swn_entries = {}

with open(swn_path, 'r') as file:
    for line in file:
        if line.startswith('#'):
            continue
        fields = line.strip().split('\t')
        if len(fields) >= 6:
            pos, synset_id, pos_score, neg_score, synset_terms, gloss = fields[:6]
            synset_terms = synset_terms.split()
            for term in synset_terms:
                term = term.split('#')[0]
                key = (pos, synset_id, term)
                value = (float(pos_score), float(neg_score))
                if key in swn_entries:
                    swn_entries[key].append(value)
                else:
                    swn_entries[key] = [value]

# Print the first two keys and their corresponding values
for key in list(swn_entries.keys())[:2]:
    print(key, swn_entries[key])


('a', '00001740', 'able') [(0.125, 0.0)]
('a', '00002098', 'unable') [(0.0, 0.75)]


In [17]:
print(len(swn_entries))

206941


In [19]:
import nltk
nltk.download('stopwords')


# Preprocess and tokenize the text data
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Join the tokens back into a single string
    preprocessed_text = ' '.join(filtered_tokens)
    
    return preprocessed_text

# Perform sentiment analysis for each book
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.txt'):
        with open(file_path, 'r') as f:
            text = f.read()
            preprocessed_text = preprocess_text(text)
            

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
import nltk

# Download the SentiWordNet resource
nltk.download('sentiwordnet')

from nltk.corpus import sentiwordnet as swn
nltk.download('wordnet')

# Perform sentiment analysis
def analyze_sentiment(text):
    sentiment_score = 0.0
    tokens = nltk.word_tokenize(text)
    for token in tokens:
        synsets = list(swn.senti_synsets(token))
        if synsets:
            synset = synsets[0]  # Consider the first synset for simplicity
            sentiment_score += synset.pos_score() - synset.neg_score()
    return sentiment_score

# Analyze the sentiment of each text
for i in range(len(texts)):
    text = texts[i]
    label = labels[i]
    
    # Perform sentiment analysis
    sentiment_score = analyze_sentiment(text)
    
    # Print the sentiment analysis results
    print(f"Text: {text[:50]}...")
    print(f"Label: {label}")
    print(f"Sentiment Score: {sentiment_score}")
    print()

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


Text: / 




THE BOY WHO LIVED 

Mr. and Mrs. Dursley, o...
Label: 1
Sentiment Score: 123.888

Text: 
J . K . R O W L ! N G 

HARRY 

POTTER 




/ 


...
Label: 2
Sentiment Score: -197.892

Text: / 




OWL POST 

Harry Potter was a highly unusua...
Label: 3
Sentiment Score: 130.334

Text: / 




THE RIDDLE HOUSE 

The villagers of Little ...
Label: 4
Sentiment Score: 551.8669999999998

Text: HARRY 

POTTER 




I 




DUDLEY DEMENTED 

The h...
Label: 5
Sentiment Score: 229.48399999999992

Text: 
* J 



/ 




THE OTHER MINISTER 

It was nearin...
Label: 6
Sentiment Score: 170.189

Text: I 



* 

THE DARK LORD ASCENDING 

The two men ap...
Label: 7
Sentiment Score: -580.322



IndexError: ignored